In [1]:
#импорт библиотек, загрузка модели, весов, разметки клаассов
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2648).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./cl_classes1307.npy', allow_pickle=True)
model.load_state_dict(torch.load("../../best_model2306/pytorch_model10d.bin", map_location=device))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [2]:
#импорт описания классов (Алексей слямзил)
dscr = pd.read_csv("../data/descr10d.csv", sep=';', names=['id', 'label', 'till'], dtype={'id': str, 'label': str, 'till': str})

In [101]:
#dscr['till'] = pd.to_datetime(dscr['till'], format='%Y-%m-%d')
#dscr['label'] = dscr['label'].str.lower()
#dscr['label'] = dscr['label'].str.strip()
#dscr.to_csv('../data/descr10d.csv', sep=';', columns=['id', 'label', 'till'], index=False, header=False)

In [3]:
dscr['valid'] = dscr['till'].isna()

In [4]:
#Устаревшие коды
dscr[dscr['till'].notna()]

,id,label,till,valid
85,8421990008,прочие части оборудования и устройств для филь...,2021-12-31,False
99,7019590000,ткани прочие из стекловолокна,2021-12-31,False
251,7019900009,стекловолокно (включая стекловату) и изделия и...,2021-12-31,False
406,8479899708,"прочие машины и механические приспособления, в...",2021-12-31,False
467,9405109803,прочие люстры и прочее электрическое осветитие...,2021-12-31,False
...,...,...,...,...
2625,7019390008,"прочие холсты, маты, матрицы, плиты и прочие н...",2021-12-31,False
2626,8474100000,"машины для сортировки, грохочения, сепарации и...",2021-04-02,False
2627,7323999100,"прочие столовые изделия из черного металла, ла...",2016-12-31,False
2629,1515904000,"прочие масла и их фракции сырые, нерафинирован...",2021-12-31,False


In [7]:
#точки данных с устаревшими кодами
df[df['id'].isin(dscr[dscr['till'].notna()]['id'])]

,id,label
63003,1207999600,семена чиа шалфей испанский лат salvia hispani...
63004,1207999600,семена тыквы свежие очищенные для продовольств...
63005,1207999600,семена масличной тыквы cucurbita pero styriaca...
63006,1207999600,семена тыквы сырые лущенные для употребления в...
63007,1207999600,семена тыквы сырые недробленые очищенные от ко...
...,...,...
232945,7318159009,прочие винты и болты с головками из черных мет...
232947,8421990008,запасные части принадлежности и компоненты тра...
232951,9403901000,части мебели из металла изделие сложной геомет...
232960,9405105008,светильники электрические поставляются частичн...


In [8]:
#функция предсказания одного класса (аргмакс). По умолчанию с кодом класса дает описание класса
def predict_class(text, desc=True):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        if desc==True:
            result = dict()
            id = Label_encoder.inverse_transform([predicted_class_id])[0]
            result[id] = dscr[dscr['id']==id].iloc[0]['label']
            return result
        else:
            return Label_encoder.inverse_transform([predicted_class_id])[0]

In [24]:
#Пример с описанием
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО')

{'1704906500': 'Кондитерские изделия в виде резинки и желе, включая фруктовую пасту в виде кондидерских изделий из сахара, не содержащие какао'}

In [5]:
#Пример без описания
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО', desc=False)

'1704906500'

In [9]:
#функция предикта классов с вероятностью (софтмакс), по умолчанию 5 классов
def predict_prob(text, qtty=5):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    result = dict()
    p = torch.nn.functional.softmax(logits, dim=1)
    for i in range(qtty):
        a = p.argmax().item()
        result[Label_encoder.inverse_transform([a])[0]] = p[0][a].item()
        p[0][a] = 0
    return result

In [10]:
#функция предикта классов и описания с вероятностью, по умолчанию 5 шт. Использует функцию predict_prob.
#возвращает список списков
def predict_prob_with_descr(text, qtty=5):
    probs = predict_prob(text, qtty=qtty)
    #result = np.array()
    result = list()
    for each in probs:
        #valid = np.nonzero(dscr[dscr['id']==each].iloc[0]['till'])
        
        result.append([each, dscr[dscr['id']==each].iloc[0]['label'], probs[each], dscr[dscr['id']==each].iloc[0]['valid']])
    return result

In [13]:
#Пример с 3 классами
print(predict_prob_with_descr('шины пневматические резиновые новые', 3))

[['4011800000', 'шины и покрышки пневматические резиновые новые для транспортных средств и машин, используемых в строительстве, горном деле или промышленности', 0.9863451719284058, True], ['4011209000', 'шины и покрышки пневматические резиновые новые, для автобусов или моторных транспортных средств для перевозки грузов с индексом нагрузки более 121', 0.0040657068602740765, True], ['4011900000', 'прочие шины и покрышки пневматические резиновые новые', 0.0034278009552508593, True]]


In [137]:
#Пример с 5 классами
print(predict_prob_with_descr('семена тыквы ', 5))

[['1207999600', 'семена и плоды прочих масличных культур, дробленые или недробленые, прочие', 0.9361142516136169, False], ['2008199900', 'прочие орехи, арахис и прочие семена, смешанные или не смешанные между собой в первичных упаковках нетто-массой не более 1 кг', 0.010454857721924782, True], ['1207409000', 'прочие семена кунжута, дробленые или недробленые', 0.00979942362755537, True], ['0711400000', 'огурцы и корнишоны, консервированные для кратковременного хранения (например, диоксидом серы, в рассоле, сернистой воде или в другом временно консервирующем растворе), но в таком виде непригодные для непосредственного употребления в пищу', 0.0006681593949906528, True], ['2001902000', 'плоды рода capsicum, кроме перца стручкового сладкого или перца душистого, приготовленные или консервированные с добавлением уксуса или уксусной кислоты', 0.0006370716728270054, True]]


In [110]:
#причесанный в пандас пример
pd.DataFrame(predict_prob_with_descr('кофе арабика'), columns=['ID', 'LABEL', 'PROB', 'TILL'])

,ID,LABEL,PROB,TILL
0,0901210001,кофе жареный с кофеином в зернах: арабика (cof...,0.413376,NaN
1,2101129201,"готовые продукты на основе экстратов, эссенций...",0.255713,NaN
2,0901210009,кофе жареный с кофеином прочий,0.133681,NaN
3,0901220009,кофе жареный без кофеина прочий,0.019602,NaN
4,1704909800,прочие кондитерские изделия из сахара (включая...,0.005251,NaN


In [112]:
def consolelog(probs):
    print('prob-ty', 'Code', 'Description', sep=':    ')
    #print('____________________________________________________________________')
    for each in probs:
        if len(each[1]) < 40:
            des = each[1]
        else:
            des = each[1][:40] + '...'
        print(f"{each[2]:.{5}f}", each[0], des, sep=':    ')
        #print('____________________________________________________________________')

In [113]:
consolelog(predict_prob_with_descr("аттракционы", 5))

prob-ty:    Code:    Description
0.00686:    8714993000:    багажники к транспортным средствам товар...
0.00610:    8541401000:    светоизлучающие диоды (led), включая лаз...
0.00571:    9508900000:    прочие карусели, качели, тиры и прочие а...
0.00551:    8529906509:    электронные модули, предназначенные искл...
0.00536:    9508260000:    аттракционы водных парков


In [6]:
#загрузим обучающий датасет
df = pd.read_csv("../data/mergedcleared1307.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#df = pd.read_csv("../data/firstset.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#функция для вывода обучающих данных по коду. При pandas=False возвращает numpy массив
def whats_data(id, pandas=True):
    if pandas == True:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)]
    else:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)].to_numpy()

In [12]:
#пример использования по полному коду (можно делать по части кода)
whats_data('4011100009', pandas=False)

array([['4011100009',
        'шины пневматические резиновые новые для легковых автомобилей с посадочным диаметром более 16 дюймов размер модель индекс нагрузки категория скорости артикул кол во единиц 205 40zr17 atrezzo zsr 84y xl 3220010146 4 шт'],
       ['4011100009',
        'шины пневматические резиновые новые для легковых автомобилей с посадочным диаметром более 16 дюймов размер модель индекс нагрузки категория скорости артикул кол во единиц 215 45r17 ice blazer wst3 91t xl 3220006483 40 шт'],
       ['4011100009',
        'шины пневматические резиновые новые для легковых автомобилей с посадочным диаметром более 16 дюймов размер модель индекс нагрузки категория скорости артикул кол во единиц 215 50r17 ice blazer wst3 95t xl 3220006181 261 шт'],
       ...,
       ['4011100009',
        'шины пневматические резиновые новые для легкогрузовых автомобилей с посадочным диаметром более 16 дюймов торговой марки sailun размер модель индекс нагрузки категория скорости кол во единиц 245 4

In [14]:
#еще пример
df[df['id'].isin(list(whats_data('4011100009')['id'].unique()))]

,id,label
135406,4011100009,шины пневматические резиновые новые для легков...
135407,4011100009,шины пневматические резиновые новые для легков...
135408,4011100009,шины пневматические резиновые новые для легков...
135409,4011100009,шины пневматические резиновые новые для легков...
135410,4011100009,шины пневматические резиновые новые для легков...
...,...,...
177887,4011100009,шины пневматические резиновые новые для легког...
177888,4011100009,шины пневматические резиновые новые для легког...
177889,4011100009,шины пневматические резиновые новые для легког...
177890,4011100009,шины пневматические резиновые новые для легког...


In [11]:
similar = pd.read_csv("../data/Similar.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
similar

,id,label
0,4011100009,шины
1,4011100009,шина
2,4011100009,авто шины
3,4011100009,чернитель шин
4,4011100009,тапочки резиновые
...,...,...
59752,8474100000,грохот sandvik
59753,8474100000,вибрационный стол
59754,8474100000,вибрационные грохоты для горнодобывающей промы...
59755,8474100000,аппарат для деления руды
